In [1]:
import ast
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import multirecording_spikeanalysis as spike
from scipy.stats import chi2_contingency
from pathlib import Path
from collections import defaultdict

In [2]:
# Load the DataFrame using pandas
rce3_alone_clusters = pd.read_pickle("rce_pilot_3_alone_comp_cluster_ranges.pkl")

# Initialize the dictionary
timestamp_dicts = {}

# Loop through each row of the DataFrame to populate the dictionary
for index, row in rce3_alone_clusters.iterrows():
    # Define the key as a combination of session_dir and current_subject
    key = f"{row['session_dir']}"

    start_pos = key.find('subj_')
    # Add the length of 'subj_' and 3 additional characters to include after 'subj_'
    end_pos = start_pos + len('subj_') + 3
    # Slice the recording key to get everything up to and including the subject identifier plus three characters
    key_without_suffix = key[:end_pos]
    
    # Initialize nested dictionary for this key if not already present
    if key not in timestamp_dicts:
        timestamp_dicts[key_without_suffix] = {}

    # Loop through the cluster_timestamps_ranges_dict to populate conditions
    for condition, ranges in row['cluster_timestamps_ranges_dict'].items():
        # Check if the condition key already exists, if not initialize a list
        if condition not in timestamp_dicts[key_without_suffix]:
            timestamp_dicts[key_without_suffix][str(condition)] = []

        # Extend the existing list with new ranges
        modified_ranges = [(start // 20, end // 20) for start, end in ranges]
        timestamp_dicts[key_without_suffix][str(condition)].extend(modified_ranges)

# Optionally, convert the lists to numpy arrays as in your previous code
for subj_recording in timestamp_dicts:
    for condition in timestamp_dicts[subj_recording]:
        timestamp_dicts[subj_recording][condition] = np.array(timestamp_dicts[subj_recording][condition], dtype=np.int64)

In [3]:
def combine_intervals(ranges):
    """Combine intervals that are within 250 milliseconds of each other."""
    if ranges.size == 0:
        return ranges
    # Sort ranges based on the start times
    sorted_ranges = np.array(sorted(ranges, key=lambda x: x[0]))
    combined = [sorted_ranges[0]]

    for current in sorted_ranges[1:]:
        last = combined[-1]
        # If the current start is within 250 ms of the last end, combine them
        if current[0] - last[1] <= 250:
            combined[-1] = [last[0], max(last[1], current[1])]
        else:
            combined.append(current)

    return np.array(combined)

def remove_short_intervals(ranges):
    """Remove intervals that are less than 250 milliseconds long."""
    return np.array([interval for interval in ranges if interval[1] - interval[0] >= 250])

def process_timestamps_nested(timestamp_dicts):
    timestamp_dicts_cut = {}
    for date_key, clusters in timestamp_dicts.items():
        timestamp_dicts_cut[date_key] = {}
        for cluster_key, intervals in clusters.items():
            intervals = np.array(intervals)  # Ensure intervals is a numpy array
            if intervals.size > 0 and intervals.ndim == 2:  # Check if there are any intervals and it's 2-dimensional
                combined = combine_intervals(intervals)
                cleaned = remove_short_intervals(combined)
                timestamp_dicts_cut[date_key][cluster_key] = cleaned
            else:
                timestamp_dicts_cut[date_key][cluster_key] = np.array([])  # Handle empty or malformed input case
    return timestamp_dicts_cut

# Assuming timestamp_dicts is defined and filled with your data
timestamp_dicts_cut = process_timestamps_nested(timestamp_dicts)

In [4]:
def find_next_window(start_time, all_timestamps, all_ranges, gap):
    # Find the next start time that is greater than the given start_time
    next_start = min([t for t in all_timestamps if t > start_time], default=None)
    if next_start is None:
        return None, None
    
    # Determine the maximum end time that doesn't exceed next_start + gap
    potential_ends = [end for start, end in all_ranges if start >= next_start and end <= next_start + gap]
    if not potential_ends:
        return next_start, next_start + gap  # Extend to the full window length if no end within range
    next_end = max(potential_ends)
    return next_start, next_end

# Process each recording in timestamp_dicts_cut
windowed_events = {}

for recording_key, clusters in timestamp_dicts_cut.items():
    # Prepare data arrays
    all_ranges = []
    all_cluster_ids = []
    for cluster_id, times in clusters.items():
        for start_end in times:
            all_ranges.append(start_end)
            all_cluster_ids.append(cluster_id)
    
    all_starts = np.array([r[0] for r in all_ranges])
    all_ends = np.array([r[1] for r in all_ranges])
    
    # Sort ranges by start times for processing order
    sorted_indices = np.argsort(all_starts)
    all_starts = all_starts[sorted_indices]
    all_ends = all_ends[sorted_indices]
    all_cluster_ids = [all_cluster_ids[i] for i in sorted_indices]
    
    # Initialize windows
    min_timestamp = all_starts.min()
    dynamic_windows = [(min_timestamp, min(min_timestamp + 30000, all_ends.max()))]
    
    # Find subsequent windows
    current_end = dynamic_windows[-1][1]
    while True:
        next_start, next_end = find_next_window(current_end, all_starts, list(zip(all_starts, all_ends)), 30100)
        if next_start is None:
            break
        dynamic_windows.append((next_start, next_end))
        current_end = next_end

    # Map clusters to windows
    windows_dict = {}
    for window_num, (window_start, window_end) in enumerate(dynamic_windows):
        clusters_in_window = []
        for idx, (start, end) in enumerate(zip(all_starts, all_ends)):
            if start >= window_start and end <= window_end and all_cluster_ids[idx] != '4':  # The last and statement removes cluster '4' because only 1 occurs so far
                clusters_in_window.append(all_cluster_ids[idx])
        
        if clusters_in_window:  # Only store windows with data
            windows_dict[f"Window {window_num + 1}"] = clusters_in_window

    windowed_events[recording_key] = windows_dict

In [15]:
# Assume timestamp_dicts_cut is already defined and loaded with data
timestamps = timestamp_dicts_cut['20240320_171038_alone_comp_subj_4-2']

# Initialize an empty list to collect all timestamp pairs
all_timestamps = []

# Iterate over each key in the dictionary and extend the list with the arrays
for key in timestamps.keys():
    all_timestamps.extend(timestamps[key])

# Sort all timestamp pairs based on the first element of each pair
all_timestamps_sorted = sorted(all_timestamps, key=lambda x: x[0])

# Convert the sorted list of pairs into a numpy array
combined_sorted_array = np.array(all_timestamps_sorted)

# Now combined_sorted_array is a single long array of all sorted timestamps
print(combined_sorted_array)

[[1703995 1704688]
 [1704965 1706906]
 [1707114 1709123]
 ...
 [3074205 3080441]
 [3080510 3081480]
 [3081480 3083074]]


In [17]:
combined_sorted_array

array([[1703995, 1704688],
       [1704965, 1706906],
       [1707114, 1709123],
       ...,
       [3074205, 3080441],
       [3080510, 3081480],
       [3081480, 3083074]])

In [21]:
combined_sorted_array[0].shape

(2,)

In [23]:
# Initialize an empty list to collect the differences
differences = []

# Iterate through the array of pairs to calculate the differences
for i in range(len(combined_sorted_array) - 1):
    current_end = combined_sorted_array[i][1]
    next_start = combined_sorted_array[i + 1][0]
    difference = next_start - current_end

    # Only include differences less than 29,000
    if difference < 30100:
        differences.append(difference)

# Convert the list of valid differences into a numpy array
valid_differences_array = np.array(differences)

# Now valid_differences_array contains the required differences
print(valid_differences_array)

[  277   208   208     0    70    69   278    69   346    70    70    69
    70    36     0    70    69   347     0    69 20996    69    69    69
     0    53    70    69     0    69     9     0    70     0    69    70
    70     0     0   347    69    41    69    70 25985   277    69    69
   278    69    69   277    44     0    69    70     0   208    69    70
    70    70   555   347    31    70     5    69    69     0     0    69
   277   208    69    70    69 21204    70    69    51    69    69   346
    69     0    69     0    70    12    69   347    69     0     0   139
    69    69   555    70     0    69    69    69    69    69    69    69
    70    69     0   555    69     0    69    70    70     0     0    69
     2     0    70    69    69     0    70    34     0    70    70     0
    70    69    69    70    70    69    69    70     0    70    69    69
    44   139   208     0   139   208    69    69    49    69   138    52
    69     0    69    69    69    69   508   346   

In [24]:
def find_next_window(start_time, all_timestamps, all_ranges, gap):
    # Debugging addition: Output the current start time and the computation of the next start
    print(f"Finding next window from start time: {start_time}")
    next_starts = [t for t in all_timestamps if t > start_time]
    if not next_starts:
        print("No more starts found.")
        return None, None
    
    next_start = min(next_starts)
    print(f"Next start found at: {next_start}")
    
    # Determine the maximum end time that doesn't exceed next_start + gap
    potential_ends = [end for start, end in all_ranges if start >= next_start and end <= next_start + gap]
    if not potential_ends:
        print("No ends found within the window range, using default window length.")
        return next_start, next_start + gap
    
    next_end = max(potential_ends)
    print(f"Next end within window range found at: {next_end}")
    return next_start, next_end

# The rest of the script for processing would follow here, ensuring all debug outputs
# help trace the steps and values correctly.

# Process each recording in timestamp_dicts_cut
windowed_events = {}

for recording_key, clusters in timestamp_dicts_cut.items():
    # Prepare data arrays
    all_ranges = []
    all_cluster_ids = []
    for cluster_id, times in clusters.items():
        for start_end in times:
            all_ranges.append(start_end)
            all_cluster_ids.append(cluster_id)
    
    all_starts = np.array([r[0] for r in all_ranges])
    all_ends = np.array([r[1] for r in all_ranges])
    
    # Sort ranges by start times for processing order
    sorted_indices = np.argsort(all_starts)
    all_starts = all_starts[sorted_indices]
    all_ends = all_ends[sorted_indices]
    all_cluster_ids = [all_cluster_ids[i] for i in sorted_indices]
    
    # Initialize windows
    min_timestamp = all_starts.min()
    dynamic_windows = [(min_timestamp, min(min_timestamp + 30000, all_ends.max()))]
    
    # Find subsequent windows
    current_end = dynamic_windows[-1][1]
    while True:
        next_start, next_end = find_next_window(current_end, all_starts, list(zip(all_starts, all_ends)), 30100)
        if next_start is None:
            break
        dynamic_windows.append((next_start, next_end))
        current_end = next_end

    # Map clusters to windows
    windows_dict = {}
    for window_num, (window_start, window_end) in enumerate(dynamic_windows):
        clusters_in_window = []
        for idx, (start, end) in enumerate(zip(all_starts, all_ends)):
            if start >= window_start and end <= window_end and all_cluster_ids[idx] != '4':  # The last and statement removes cluster '4' because only 1 occurs so far
                clusters_in_window.append(all_cluster_ids[idx])
        
        if clusters_in_window:  # Only store windows with data
            windows_dict[f"Window {window_num + 1}"] = clusters_in_window

    windowed_events[recording_key] = windows_dict

Finding next window from start time: 1733995
Next start found at: 1754095
Next end within window range found at: 1783059
Finding next window from start time: 1783059
Next start found at: 1809044
Next end within window range found at: 1837801
Finding next window from start time: 1837801
Next start found at: 1859005
Next end within window range found at: 1884643
Finding next window from start time: 1884643
Next start found at: 1934050
Next end within window range found at: 1963777
Finding next window from start time: 1963777
Next start found at: 2019211
Next end within window range found at: 2048800
Finding next window from start time: 2048800
Next start found at: 2134169
Next end within window range found at: 2163480
Finding next window from start time: 2163480
Next start found at: 2183991
Next end within window range found at: 2213441
Finding next window from start time: 2213441
Next start found at: 2234090
Next end within window range found at: 2263748
Finding next window from start t

In [25]:
# Initialize an empty list to collect the differences
differences = []

# Iterate through the array of pairs to calculate the differences
for i in range(len(combined_sorted_array) - 1):
    current_end = combined_sorted_array[i][1]
    next_start = combined_sorted_array[i + 1][0]
    difference = next_start - current_end

    differences.append(difference)

# Convert the list of valid differences into a numpy array
valid_differences_array = np.array(differences)

# Now valid_differences_array contains the required differences
valid_differences_array

array([  277,   208,   208,     0,    70,    69,   278,    69,   346,
          70,    70,    69,    70,    36,     0,    70,    69,   347,
           0,    69, 20996,    69,    69,    69,     0,    53,    70,
          69,     0,    69,     9,     0,    70,     0,    69,    70,
          70,     0,     0,   347,    69,    41,    69,    70, 25985,
         277,    69,    69,   278,    69,    69,   277,    44,     0,
          69,    70,     0,   208,    69,    70,    70,    70,   555,
         347,    31,    70,     5,    69,    69,     0,     0,    69,
         277,   208,    69,    70,    69, 21204,    70,    69,    51,
          69,    69,   346,    69,     0,    69,     0,    70,    12,
          69,   347,    69,     0,     0,   139,    69,    69,   555,
          70,     0,    69,    69,    69, 49407,    69,    69,    69,
          69,    70,    69,     0,   555,    69,     0,    69,    70,
          70,     0,     0,    69,     2,     0,    70,    69,    69,
           0, 55434,

In [27]:
# Initialize an empty list to collect the window lengths
window_lengths = []

# Iterate through the dynamic_windows list to calculate lengths
for start_time, end_time in dynamic_windows:
    window_length = end_time - start_time
    window_lengths.append(window_length)

# Print the window lengths
print(window_lengths)

# Convert the list of lengths into a numpy array if needed for further analysis
window_lengths_array = np.array(window_lengths)
print(window_lengths_array)

[np.int64(30000), np.int64(28756), np.int64(29866), np.int64(29311), np.int64(28757), np.int64(29242), np.int64(29034), np.int64(29173), np.int64(29588), np.int64(29519), np.int64(29727), np.int64(28896), np.int64(29450), np.int64(29242), np.int64(27579), np.int64(28480), np.int64(26590), np.int64(28341), np.int64(28410)]
[30000 28756 29866 29311 28757 29242 29034 29173 29588 29519 29727 28896
 29450 29242 27579 28480 26590 28341 28410]


In [28]:
dynamic_windows

[(np.int64(1809114), np.int64(1839114)),
 (np.int64(1858243), np.int64(1886999)),
 (np.int64(1933218), np.int64(1963084)),
 (np.int64(2018241), np.int64(2047552)),
 (np.int64(2133199), np.int64(2161956)),
 (np.int64(2183229), np.int64(2212471)),
 (np.int64(2233190), np.int64(2262224)),
 (np.int64(2298533), np.int64(2327706)),
 (np.int64(2358542), np.int64(2388130)),
 (np.int64(2468510), np.int64(2498029)),
 (np.int64(2533300), np.int64(2563027)),
 (np.int64(2588249), np.int64(2617145)),
 (np.int64(2663225), np.int64(2692675)),
 (np.int64(2713324), np.int64(2742566)),
 (np.int64(2769521), np.int64(2797100)),
 (np.int64(2818234), np.int64(2846714)),
 (np.int64(2888498), np.int64(2915088)),
 (np.int64(2998259), np.int64(3026600)),
 (np.int64(3053209), np.int64(3081619))]